<a href="https://colab.research.google.com/github/HasibAlMuzdadid/Machine-Learning-and-Deep-Learning-Projects/blob/main/hand%20sign%20classification%5Bresnet%5D/hand_sign_classification%5Bresnet%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hand Sign Classifier using Residual Network[ResNet]**

In [ ]:
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity

In [ ]:
def load_dataset():
    train_dataset = h5py.File("train_signs.h5", "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])  # train set features
    train_set_y_orig = np.array(
    train_dataset["train_set_y"][:])                              #  train set labels

    test_dataset = h5py.File("test_signs.h5", "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])     # test set features
    test_set_y_orig = np.array(
    test_dataset["test_set_y"][:])                                # test set labels

    classes = np.array(test_dataset["list_classes"][:])           # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

**Identity Block**

Implementing the ResNet identity block. First, We can read these docs carefully to make sure understanding what's happening. Then, implement the rest. 
- To implement the Conv2D step: [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- To implement BatchNorm: [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) `BatchNormalization(axis = 3)(X, training = training)`. If training is set to False, its weights are not updated with the new examples. I.e when the model is used in prediction mode.
- For the activation, use:  `Activation('relu')(X)`
- To add the value passed forward by the shortcut: [Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

We have added the initializer argument to our functions. This parameter receives an initializer function like the ones included in the package [tensorflow.keras.initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers) or any other custom initializer. By default it will be set to [random_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomUniform)



In [ ]:
# Identity Block

def identity_block(X, f, filters, training = True, initializer = random_uniform):
    
    # X --> input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    # f --> integer, specifying the shape of the middle CONV's window for the main path
    # filters --> python list of integers, defining the number of filters in the CONV layers of the main path
    # training --> True: Behave in training mode ; False: Behave in inference mode            
    # initializer --> to set up the initial weights of a layer. Equals to random uniform initializer
    
    # Returns:
    # X --> output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value, we'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = "valid", kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)     # Default axis
    X = Activation("relu")(X)

    # Set the padding = 'same'
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding="same",kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation("relu")(X)
 
    # Set the padding = 'valid'
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding="valid", kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    # Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X_shortcut,X])
    X = Activation("relu")(X)


    return X

**Convolutional Block**

The ResNet "convolutional block" is the second block type. We can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path.

* The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. 

* The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step. 
* The additional `initializer` argument is required for grading purposes, and it has been set by default to [glorot_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform)

In [ ]:
# Convolutional Block

def convolutional_block(X, f, filters, s = 2, training=True, initializer = glorot_uniform):

    # X --> input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    # f --> integer, specifying the shape of the middle CONV's window for the main path
    # filters --> python list of integers, defining the number of filters in the CONV layers of the main path
    # s --> Integer, specifying the stride to be used
    # training --> True: Behave in training mode ; False: Behave in inference mode            
    # initializer --> to set up the initial weights of a layer. Equals to Glorot uniform initializer, also called Xavier uniform initializer.
    
    # Returns:
    # X --> output of the convolutional block, tensor of shape (n_H, n_W, n_C)
  
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation("relu")(X)

    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation("relu")(X) 

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training) 
    

    ##### SHORTCUT PATH ##### 
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    

    # Add shortcut value to main path and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

**ResNet Model (50 layers)**

We now have the necessary blocks to build a very deep ResNet.

The details of this ResNet-50 model are:
- Zero-padding pads the input with a pad of (3,3)
- Stage 1:
    - The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). 
    - BatchNorm is applied to the 'channels' axis of the input.
    - ReLU activation is applied.
    - MaxPooling uses a (3,3) window and a (2,2) stride.
- Stage 2:
    - The convolutional block uses three sets of filters of size [64,64,256] "f" is 3 and "s" is 1.
    - The 2 identity blocks use three sets of filters of size [64,64,256] and "f" is 3.
- Stage 3:
    - The convolutional block uses three sets of filters of size [128,128,512] "f" is 3 and "s" is 2.
    - The 3 identity blocks use three sets of filters of size [128,128,512] and "f" is 3.
- Stage 4:
    - The convolutional block uses three sets of filters of size [256, 256, 1024] "f" is 3 and "s" is 2.
    - The 5 identity blocks use three sets of filters of size [256, 256, 1024] and "f" is 3.
- Stage 5:
    - The convolutional block uses three sets of filters of size [512, 512, 2048] "f" is 3 and "s" is 2.
    - The 2 identity blocks use three sets of filters of size [512, 512, 2048] and "f" is 3.
- The 2D Average Pooling uses a window of shape (2,2).
- The 'flatten' layer doesn't have any hyperparameters.
- The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation.

    


In [ ]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):

    # Stage-wise implementation of the architecture of the popular ResNet50:
    # CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3 -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    # input_shape --> shape of the images of the dataset
    # classes --> integer, number of classes

    # Returns:
    # model --> a Model() instance in Keras

    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    
    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128,128,512], s = 2)
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    X = identity_block(X, 3,  [128,128,512])
    
    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024]) 

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048]) 

    # AVGPOOL 
    X = AveragePooling2D((2, 2))(X)


    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                   

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255
X_test = X_test_orig / 255

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print (f"number of training examples = {X_train.shape[0]}")
print (f"number of test examples = {X_test.shape[0]}")
print (f"X_train shape: {X_train.shape}")
print (f"Y_train shape: {Y_train.shape}")
print (f"X_test shape: {X_test.shape}")
print (f"Y_test shape: {Y_test.shape}")

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [ ]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
34/34 [==============================] - 181s 5s/step - loss: 1.8921 - accuracy: 0.4815
Epoch 2/10
34/34 [==============================] - 169s 5s/step - loss: 0.6499 - accuracy: 0.7917
Epoch 3/10
34/34 [==============================] - 167s 5s/step - loss: 0.4442 - accuracy: 0.8657
Epoch 4/10
34/34 [==============================] - 165s 5s/step - loss: 0.3611 - accuracy: 0.8852
Epoch 5/10
34/34 [==============================] - 164s 5s/step - loss: 0.2410 - accuracy: 0.9213
Epoch 6/10
34/34 [==============================] - 163s 5s/step - loss: 0.1001 - accuracy: 0.9667
Epoch 7/10
34/34 [==============================] - 164s 5s/step - loss: 0.2157 - accuracy: 0.9491
Epoch 8/10
34/34 [==============================] - 165s 5s/step - loss: 0.1316 - accuracy: 0.9565
Epoch 9/10
34/34 [==============================] - 164s 5s/step - loss: 0.1364 - accuracy: 0.9556
Epoch 10/10
34/34 [==============================] - 163s 5s/step - loss: 0.1821 - accuracy: 0.9491


Let's see how this model (trained on only two epochs) performs on the test set.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print (f"Loss = {preds[0]}")
print (f"Test Accuracy = {preds[1]}")

4/4 [==============================] - 3s 458ms/step - loss: 0.2508 - accuracy: 0.9417
Loss = 0.2508190870285034
Test Accuracy = 0.9416666626930237
